In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [2]:
# Load the dataset
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

In [3]:
# Preprocessing
categorical_features = ['쇼핑몰 구분', '도시 유형', '지역 유형', '쇼핑몰 유형', '선물 유형']
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features),
        ('num', StandardScaler(), ['가격(원)'])
    ])

In [4]:
max_weeks = train_data['추석까지 남은 기간(주)'].max()
train_data['sin_time'] = np.sin(2 * np.pi * train_data['추석까지 남은 기간(주)'] / max_weeks)
train_data['cos_time'] = np.cos(2 * np.pi * train_data['추석까지 남은 기간(주)'] / max_weeks)
train_data['log_수요량'] = np.log1p(train_data['수요량'])

In [13]:
# Preprocess using OneHotEncoder and StandardScaler
X_processed = preprocessor.fit_transform(train_data.drop(['ID', '수요량', 'log_수요량'], axis=1))

# Check if X_processed is a sparse matrix and convert to dense if necessary
if hasattr(X_processed, "toarray"):
    X_processed = X_processed.toarray()

# Additional time-based features
sin_time = np.sin(2 * np.pi * train_data['추석까지 남은 기간(주)'] / max_weeks).values.reshape(-1, 1)
cos_time = np.cos(2 * np.pi * train_data['추석까지 남은 기간(주)'] / max_weeks).values.reshape(-1, 1)

# Concatenate the processed features with sin_time and cos_time
X_final = np.concatenate([X_processed, sin_time, cos_time], axis=1)

# Convert processed numpy arrays to PyTorch tensors
X = torch.tensor(X_final, dtype=torch.float32)
y = torch.tensor(train_data['log_수요량'].values, dtype=torch.float32)


In [14]:
# Define custom dataset
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [15]:
# Split the data into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(X))
val_size = len(X) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(TabularDataset(X, y), [train_size, val_size])

In [16]:
# Define dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [17]:
# Define the Neural Network with Dropout
class NeuralNet(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.layer4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.dropout(x)
        x = F.relu(self.layer2(x))
        x = self.dropout(x)
        x = F.relu(self.layer3(x))
        x = self.dropout(x)
        x = self.layer4(x)
        return x

In [18]:
model = NeuralNet(X.size(1))
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
# Initialize optimizer and scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adjust lr if needed
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

In [28]:
# Training the Model with Early Stopping
num_epochs = 1000
patience = 200
best_val_loss = float('inf')
early_stopping_counter = 0

In [29]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation loss for early stopping and RMSE
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            loss = criterion(outputs.squeeze(), batch_y)
            val_loss += loss.item()
    
    rmse = torch.sqrt(torch.tensor(val_loss / len(val_loader)))

    # Check early stopping condition
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_loss/len(train_loader)}, Validation Loss: {val_loss/len(val_loader)}, Validation RMSE: {rmse.item()}")
    
    # Use the ReduceLROnPlateau scheduler
    scheduler.step(val_loss)

    if early_stopping_counter == patience:
        print("Early stopping triggered!")
        break

Epoch 1/1000, Training Loss: 0.09620553174534359, Validation Loss: 0.3534029416347805, Validation RMSE: 0.5944770574569702
Epoch 2/1000, Training Loss: 0.09584159821875997, Validation Loss: 0.35218285024166107, Validation RMSE: 0.5934499502182007
Epoch 3/1000, Training Loss: 0.093615988118423, Validation Loss: 0.35713886430389, Validation RMSE: 0.5976109504699707
Epoch 4/1000, Training Loss: 0.09507801776399484, Validation Loss: 0.35285378051431554, Validation RMSE: 0.5940149426460266
Epoch 5/1000, Training Loss: 0.09208029458248937, Validation Loss: 0.3563350360644491, Validation RMSE: 0.5969380736351013
Epoch 6/1000, Training Loss: 0.09657796255841448, Validation Loss: 0.3561962552760777, Validation RMSE: 0.5968217849731445
Epoch 7/1000, Training Loss: 0.09527673934762543, Validation Loss: 0.34959709173754644, Validation RMSE: 0.5912673473358154
Epoch 8/1000, Training Loss: 0.09438383574219975, Validation Loss: 0.3503362277620717, Validation RMSE: 0.5918920636177063
Epoch 9/1000, Tra

In [32]:
# Add sin_time and cos_time features to the test data
test_data['sin_time'] = np.sin(2 * np.pi * test_data['추석까지 남은 기간(주)'] / max_weeks)
test_data['cos_time'] = np.cos(2 * np.pi * test_data['추석까지 남은 기간(주)'] / max_weeks)

# Preprocess using the preprocessor we fit earlier on the training data
X_test_processed = preprocessor.transform(test_data.drop('ID', axis=1))

# Convert the sparse matrix to a dense array
X_test_dense = X_test_processed.toarray()

# Extract sin_time and cos_time and reshape for concatenation
sin_time_test = test_data['sin_time'].values.reshape(-1, 1)
cos_time_test = test_data['cos_time'].values.reshape(-1, 1)

# Concatenate
X_test_final = np.hstack([X_test_dense, sin_time_test, cos_time_test])

# Convert to PyTorch tensor
X_test_tensor = torch.tensor(X_test_final, dtype=torch.float32)


In [34]:
with torch.no_grad():
    test_predictions = model(X_test_tensor)
test_predictions = np.expm1(test_predictions.numpy().squeeze())

In [35]:
# Create Submission File
submission = pd.DataFrame({
    'ID': test_data['ID'],
    '수요량': test_predictions
})
submission.to_csv('submission.csv', index=False)